In [20]:
import numpy as np, pandas as pd, polars as pl
from sklearn.model_selection import KFold
from sklearn import preprocessing as skp
from sklearn.decomposition import PCA 
import seaborn as sns

import xgboost as xgb
from sklearn.metrics import root_mean_squared_error as rmse
import matplotlib.pyplot as plt

In [21]:
train = pd.read_csv('train.csv', index_col = 'id')
test = pd.read_csv('test.csv', index_col = 'id')

In [22]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 750000 entries, 0 to 749999
Data columns (total 11 columns):
 #   Column                       Non-Null Count   Dtype  
---  ------                       --------------   -----  
 0   Podcast_Name                 750000 non-null  object 
 1   Episode_Title                750000 non-null  object 
 2   Episode_Length_minutes       662907 non-null  float64
 3   Genre                        750000 non-null  object 
 4   Host_Popularity_percentage   750000 non-null  float64
 5   Publication_Day              750000 non-null  object 
 6   Publication_Time             750000 non-null  object 
 7   Guest_Popularity_percentage  603970 non-null  float64
 8   Number_of_Ads                749999 non-null  float64
 9   Episode_Sentiment            750000 non-null  object 
 10  Listening_Time_minutes       750000 non-null  float64
dtypes: float64(5), object(6)
memory usage: 68.7+ MB


In [23]:
test.info()

<class 'pandas.core.frame.DataFrame'>
Index: 250000 entries, 750000 to 999999
Data columns (total 10 columns):
 #   Column                       Non-Null Count   Dtype  
---  ------                       --------------   -----  
 0   Podcast_Name                 250000 non-null  object 
 1   Episode_Title                250000 non-null  object 
 2   Episode_Length_minutes       221264 non-null  float64
 3   Genre                        250000 non-null  object 
 4   Host_Popularity_percentage   250000 non-null  float64
 5   Publication_Day              250000 non-null  object 
 6   Publication_Time             250000 non-null  object 
 7   Guest_Popularity_percentage  201168 non-null  float64
 8   Number_of_Ads                250000 non-null  float64
 9   Episode_Sentiment            250000 non-null  object 
dtypes: float64(4), object(6)
memory usage: 21.0+ MB


In [24]:
categorical_map = {
        'Monday': 0, 'Tuesday': 1, 'Wednesday': 2, 'Thursday': 3, 'Friday': 4, 'Saturday': 5, 'Sunday': 6, # Day

        'Morning': 0, 'Afternoon': 1, 'Evening': 2, 'Night': 3,                                            # Time

        'Negative': 0, 'Neutral': 1, 'Positive': 2,                                                        # Sentiment

        'True Crime': 0, 'Comedy': 1, 'Education': 2, 'Technology': 3, 'Health': 4, 'News': 5, 'Music': 6, # Genre
        'Sports': 7, 'Business': 8, 'Lifestyle': 9
        }

categories = ['Genre', 'Podcast_Name', 'Episode_Title', 'Publication_Day', 'Publication_Time', 'Episode_Sentiment']

In [25]:
podcast = {}
for i, n in enumerate(train['Podcast_Name'].unique()):
    podcast[n] = i

categorical_map.update(podcast)

In [26]:
train['Episode_Title'] = train['Episode_Title'].str[8:]
test['Episode_Title'] = test['Episode_Title'].str[8:]

In [27]:
train = train.replace(categorical_map).fillna(0)
test = test.replace(categorical_map).fillna(0)

In [28]:
x = train.drop('Listening_Time_minutes', axis=1)
y = train['Listening_Time_minutes']

In [29]:
poly = skp.PolynomialFeatures(interaction_only=True, include_bias=False)
x_poly = poly.fit_transform(x)
test_poly = poly.fit_transform(test)

In [30]:
cols = poly.get_feature_names_out(x.columns)
x = pd.DataFrame(x_poly, columns=cols)

test_cols = poly.get_feature_names_out(test.columns)
test = pd.DataFrame(test_poly, columns=test_cols)

In [31]:
x.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 750000 entries, 0 to 749999
Data columns (total 55 columns):
 #   Column                                                  Non-Null Count   Dtype  
---  ------                                                  --------------   -----  
 0   Podcast_Name                                            750000 non-null  float64
 1   Episode_Title                                           750000 non-null  float64
 2   Episode_Length_minutes                                  750000 non-null  float64
 3   Genre                                                   750000 non-null  float64
 4   Host_Popularity_percentage                              750000 non-null  float64
 5   Publication_Day                                         750000 non-null  float64
 6   Publication_Time                                        750000 non-null  float64
 7   Guest_Popularity_percentage                             750000 non-null  float64
 8   Number_of_Ads           

In [32]:
important_features_bygain = ['Episode_Length_minutes','Episode_Title Episode_Length_minutes', 'Episode_Title', 'Podcast_Name','Number_of_Ads',
'Episode_Length_minutes Host_Popularity_percentage', 'Podcast_Name Episode_Length_minutes', 'Host_Popularity_percentage']
idx = []
for i, col in enumerate(x.columns): 
    if col in important_features_bygain: idx.append(i)

In [33]:
x_n = x.iloc[idx]
test_n = test.iloc[idx]

In [34]:
for c in categories: 
    x_n[c] = x_n[c].astype('category')
    test_n[c] = test_n[c].astype('category')

C:\Users\evan\AppData\Local\Temp\ipykernel_23908\4176405769.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x_n[c] = x_n[c].astype('category')
C:\Users\evan\AppData\Local\Temp\ipykernel_23908\4176405769.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_n[c] = test_n[c].astype('category')


In [ ]:
K = 100
kf = KFold(K, shuffle=True, random_state=55)

In [36]:
pred = pd.DataFrame()
pred.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 0 entries
Empty DataFrame


In [ ]:
for train_idx, val_idx in kf.split(x,y):
    x_t, y_t = x.iloc[train_idx], y.iloc[train_idx]
    x_v, y_v = x.iloc[val_idx], y.iloc[val_idx]
    model = xgb.XGBRegressor(objective='reg:squarederror', eval_metric = 'rmse', enable_categorical = True, device = 'cuda', tree_method="hist", 
                             min_child_weight=4, learning_rate=0.1, n_estimators=1000, importance_type='gain')#, max_depth=5)
    model.fit(x_t, y_t, eval_set=[(x_v,y_v)])
    pred = pd.concat([pd.DataFrame(model.predict(test)), pred], axis=1)

[0]	validation_0-rmse:23.17940
[1]	validation_0-rmse:20.18729
[2]	validation_0-rmse:18.00625
[3]	validation_0-rmse:16.45673
[4]	validation_0-rmse:15.37132
[5]	validation_0-rmse:14.63072
[6]	validation_0-rmse:14.13065
[7]	validation_0-rmse:13.79995
[8]	validation_0-rmse:13.57968
[9]	validation_0-rmse:13.43296
[10]	validation_0-rmse:13.33807
[11]	validation_0-rmse:13.27384
[12]	validation_0-rmse:13.23161
[13]	validation_0-rmse:13.20060
[14]	validation_0-rmse:13.17859
[15]	validation_0-rmse:13.16286
[16]	validation_0-rmse:13.15133
[17]	validation_0-rmse:13.14117
[18]	validation_0-rmse:13.13330
[19]	validation_0-rmse:13.12737
[20]	validation_0-rmse:13.12200
[21]	validation_0-rmse:13.11854
[22]	validation_0-rmse:13.11452
[23]	validation_0-rmse:13.11305
[24]	validation_0-rmse:13.10893
[25]	validation_0-rmse:13.10649
[26]	validation_0-rmse:13.10399
[27]	validation_0-rmse:13.10240
[28]	validation_0-rmse:13.09987
[29]	validation_0-rmse:13.09978
[30]	validation_0-rmse:13.09796
[31]	validation_0-

In [38]:
pred.reset_index(drop=True, inplace=True)
pred.index += 750000
pred_sum = pred.sum(axis=1) / K

In [39]:
pred.head()

,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
750000,54.610062,53.112171,53.134163,54.093750,53.780518,52.234871,53.883133,53.565277,53.354778,53.513210,53.591190,53.863220,53.947338,52.342583,53.358822,52.905689,53.305271,54.315044,53.982105,52.997280
750001,17.726820,17.208820,17.525555,17.325138,18.169210,18.083851,18.200481,17.749542,18.403389,17.649319,18.001040,17.984846,17.734383,18.535912,17.431072,17.879128,16.975969,17.583469,17.697590,16.713053
750002,50.931431,48.732952,49.561348,51.299805,51.366325,50.051968,48.731133,49.761009,51.459396,50.398232,51.774136,48.907139,52.318840,48.858345,49.887440,48.253124,49.454506,50.592525,51.547707,49.433804
750003,76.259628,72.607437,77.092667,77.677078,72.408623,77.985092,65.771957,73.473808,82.386734,77.573441,73.076019,71.424965,71.283157,74.097427,75.840981,71.369576,77.907990,70.889359,69.428543,74.507957
750004,49.181553,49.138840,48.113251,48.730095,51.128399,48.979172,48.361408,49.313000,49.330753,48.639767,50.314713,50.208622,49.471939,48.343674,48.275135,50.029411,49.414406,48.472363,48.261154,49.554604


In [40]:
pred_sum.rename('Listening_Time_minutes', inplace=True)
pred_sum.head()

750000    53.494526
750001    17.728931
750002    50.166061
750003    74.153122
750004    49.163116
Name: Listening_Time_minutes, dtype: float32

In [ ]:
pred_sum.to_csv('submissions/submission_100fnp.csv', index_label='id')